In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

batch_size = 64

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [7]:
print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        if in_size == 1:
            transform = transforms.ToTensor()
            x = transform(x)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307646
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.302324
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.282791
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.277752
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.257084
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.248530
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.217001
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.215432
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.179551
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.089638
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.056934
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.984116
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.790244
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.587205
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.126001
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.268232
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.948109
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.882913
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.784556
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.667075
Train Epoch: 1 [12800/60000 (

<ipython-input-9-f6808daa1815>:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
<ipython-input-9-f6808daa1815>:43: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
C:\Users\teang\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [11]:
torch.save(model.state_dict(), 'PATH.pth')



In [12]:
model2 = Net()
model2.load_state_dict(torch.load('PATH.pth'))
model2.eval()


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

In [13]:
import sys
if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("main.py <img path>")
    else:
        with open(sys.argv[1]) as data:
            data = torch.utils.data.DataLoader(dataset=data, batch_size=1)
            output = model2(data)

main.py <img path>


In [14]:
from IPython.display import Image
import numpy as np
img = Image(filename='C:\\Users\\teang\PycharmProjects\C084/mnist_input_0.png')
print(type(img))

<class 'IPython.core.display.Image'>


In [15]:
from PIL import Image

image = Image.open('C:\\Users\\teang\PycharmProjects\C084\mnist_input_0.png')
print(image.size)
image=image.resize((28,28))
print(image.size)

(1300, 1300)
(28, 28)
